In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.screening_modes_api import ScreeningModesApi

from exact_sync.v1.models import ImageSet, Team, Image, ScreeningMode
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [4]:
configuration = Configuration()
configuration.username = 'marzahl'
configuration.password = '****'
configuration.host =  "https://exact.cs.fau.de/"

In [5]:
client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
images_api = ImagesApi(client)
screening_api = ScreeningModesApi(client)

In [13]:
screened_images = [
    # SREP
       '05_EIPH_569923 Berliner Blau.svs',
       '11_EIPH_575697 Berliner Blau.svs',
       '02_EIPH_574162 berliner blau.svs',
       '26_EIPH_566482 L Berliner Blau.svs',
       '31_EIPH_568354 Berliner Blau.svs',
       '04_EIPH_567017 Berliner Blau.svs',
       '27_EIPH_571557 berliner blau.svs',
       '01_EIPH_563479 Berliner Blau.svs',
       '03_EIPH_566933 R Berliner Blau neu.svs',
       '20_EIPH_576150 berliner blau.svs',
       '07_EIPH_574882 left lung Turnbull blue.svs',
       '09_EIPH_563417 L Turnbull blue.svs',
       '14_EIPH_568381 Turnbull blue.svs',
       '22_EIPH_575216 Turnbull blue.svs',
       '28_EIPH_569948 L Turnbull blue.svs',
       '29_EIPH_566481 L Turnbull blue.svs',
    # SDATA Equine
        '07_EIPH_574882 Berliner Blau-001.svs',
           '08_EIPH_574999 Berliner Blau.svs',
           '09_EIPH_563417 L berliner blau.svs',
           '10_EIPH_563417 R Berliner Blau.svs',
           '13_EIPH_570370 Berliner Blau.svs',
           '14_EIPH_568381 berliner blau-001.svs',
           '15_EIPH_568320 berliner blau.svs',
           '16_EIPH_574882 R Berliner Blau.svs',
           '17_EIPH_575796 Berliner Blau.svs',
           '18_EIPH_574882 R Berliner Blau.svs',
           '19_EIPH_566933 L Berliner Blau.svs',
           '22_EIPH_575216 Berliner Blau.svs',
           '23_EIPH_563476 Berliner Blau-001.svs',
           '24_EIPH_576255 Berliner Blau.svs',
           '25_EIPH_568150 Berliner Blau.svs',
           '28_EIPH_569948 L berliner blau.svs',
           '29_EIPH_566481 L Berliner Blau.svs',
           '30_EIPH_588355 Berliner Blau.svs',
           '01_EIPH_563479 Turnbull blue.svs',
           '02_EIPH_574162 Turnbull blue-001.svs',
           '03_EIPH_566933 R Turnbull blue.svs',
           '04_EIPH_567017 Turnbull blue-001.svs',
           '05_EIPH_569923 Turnbull blue.svs',
           '10_EIPH_563417 R Turnbull blue-002.svs',
           '11_EIPH_575697 Turnbull blue-002.svs',
           '13_EIPH_570370 Turnbull blue-001.svs',
           '15_EIPH_568320 Turnbull blue.svs',
           '16_EIPH_576172 Turnbull blue-002.svs',
           '17_EIPH_575796 Turnbull blue.svs',
           '18_EIPH_574882 Turnbull blue-002.svs',
           '19_EIPH_566933 L Tunrbull blue.svs',
           '20_EIPH_576150 Turnbull blue.svs',
           '21_EIPH_566482 R Turnbull blue.svs',
           '23_EIPH_563476 Turnbull blue.svs',
           '24_EIPH_576255 Turnbull blue.svs',
           '25_EIPH_568150 Turnbull blue.svs',
           '27_EIPH_571557 Turnbull blue.svs',
           '30_EIPH_568355 Turnbull blue.svs',
           '31_EIPH_568354 Turnbull blue.svs',
    # SDATA-Cat
'BAL_Cat_Turnbull blue_1.svs', 'BAL_Cat_Turnbull blue_2.svs',
       'BAL_Cat_Turnbull blue_6.svs', 'BAL_Cat_Turnbull blue_10.svs',
       'BAL_Cat_Turnbull blue_12.svs', 'BAL_Cat_Turnbull blue_13.svs',
       'BAL_Cat_Turnbull blue_14.svs',
    #SDATA-Human
'2702_20 BB Human BAL-001.svs', '2707_20 TB BAL Human neu.svs',
       '10052_19 humane BAL Berliner Blau-001.svs',
       '10080_19 humane BAL Berliner Blau-001.svs',
       '10120_19 humane BAL Berliner Blau.svs',
       '10227_19 humane BAL Berliner Blau.svs',
       '10672_19 humane BAL Berliner Blau.svs',
       '10672_19 humane BAL Turnbull Blau.svs',
       '10835_19 humane BAL Berliner Blau.svs',
       '10835_19 humane BAL Turnbull Blau.svs',
       '11323_19 humane BAL Berliner Blau.svs',
       '11480_19 humane BAL Berliner Blau.svs'
    
]

In [14]:
len(screened_images)

74

In [15]:
data = []
µm_pixel = 0.25

for image_set_id in tqdm([246, 245, 251, 15, 222, 247]):
    for image in images_api.list_images(pagination=False, image_set=image_set_id).results:
        
        if image.name not in screened_images:
            continue
        
        area = (image.width * image.height) / 1000000 * (µm_pixel * µm_pixel)
        
        row = None
        for screening in screening_api.list_screening_modes(image=image.id, user=12, omit="screening_tiles").results:         
            row = [image.name, image.id, image_set_id, image.height, 
                         image.width, screening.x_steps, screening.y_steps, screening.x_steps * screening.y_steps, area]
            
            break
        if row is None:
            row = [image.name, image.id, image_set_id, image.height, 
                         image.width, 0, 0, 0, area]
        
        data.append(row)
        
imageStat = pd.DataFrame(data, columns=["name", "id", "image_set", "height", "width", "x_steps", "y_steps", "patches", "area"])
imageStat    

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:15<00:00,  2.50s/it]


,name,id,image_set,height,width,x_steps,y_steps,patches,area
0,07_EIPH_574882 Berliner Blau-001.svs,3530,246,32990,37799,22,33,726,77.936813
1,08_EIPH_574999 Berliner Blau.svs,3536,246,36678,37799,22,36,792,86.649483
2,09_EIPH_563417 L berliner blau.svs,3539,246,40195,41399,24,40,960,104.002050
3,10_EIPH_563417 R Berliner Blau.svs,3543,246,37633,39599,23,37,851,93.139323
4,13_EIPH_570370 Berliner Blau.svs,3545,246,32358,34200,20,32,640,69.165225
...,...,...,...,...,...,...,...,...,...
69,24_EIPH_576255 Turnbull blue.svs,3567,247,33641,34200,14,22,308,71.907638
70,25_EIPH_568150 Turnbull blue.svs,3569,247,32365,32399,13,22,286,65.537102
71,27_EIPH_571557 Turnbull blue.svs,3572,247,31881,32399,19,32,608,64.557032
72,30_EIPH_568355 Turnbull blue.svs,3574,247,31878,34200,14,21,294,68.139225


In [16]:
imageStat["patches"].sum()

51110

In [17]:
imageStat["area"].sum()

5196.172132562499

In [18]:
imageStat.shape

(74, 9)